# RAWG API: Descarga y Análisis de Videojuegos (1995–2024)
Este notebook contiene una explicación razonada y paso a paso del script para obtener datos de videojuegos desde la API de RAWG, filtrando por puntuación y género, y guardándolos en un archivo CSV para su análisis posterior.

## 1. Importar librerías necesarias

In [1]:
import requests
import time
import csv

## 2. Definir clave de API y URL base de RAWG

In [2]:
API_KEY = '9e9ab06256174d82a01dc73f02418ca2'
BASE_URL = 'https://api.rawg.io/api/games'

## 3️. Función para obtener videojuegos año por año desde 1995 hasta 2024

In [5]:
def obtener_juegos_desde_1995(paginas_por_año=3):
    '''
    Recorre los años 1995–2024, descargando videojuegos válidos desde la API de RAWG.
    Solo incluye juegos con puntuación y al menos un género.
    '''
    juegos_por_año = []  # Lista final donde se almacenarán todos los juegos

    for año in range(1995, 2025):  # Recorrer del 1995 al 2024 (inclusive)
        juegos_este_año = []  # Lista temporal para juegos del año actual

        for pagina in range(1, paginas_por_año + 1):  # Hasta 3 páginas por año (por defecto)
            params = {
                'key': API_KEY,  # Clave personal de la API
                'dates': f'{año}-01-01,{año}-12-31',  # Filtrado por año
                'page': pagina,  # Página actual
                'page_size': 40,  # Máximo por página permitido por la API
                'ordering': '-rating'  # Orden descendente por puntuación
            }

            response = requests.get(BASE_URL, params=params)  # Hace la petición a la API

            if response.status_code != 200:
                print(f"Error {response.status_code} en el año {año}, página {pagina}")
                break  # Si hay error, salir del bucle de páginas

            data = response.json()  # Convertir respuesta en JSON

            for juego in data.get('results', []):  # Iterar sobre los resultados de juegos
                rating = juego.get('rating')
                if rating and rating > 0 and juego.get('genres'):  # Solo juegos válidos
                    juegos_este_año.append({
                        'nombre': juego.get('name'),
                        'fecha_lanzamiento': juego.get('released'),
                        'año': año,
                        'puntuacion_media': rating,
                        'generos': ', '.join([g['slug'] for g in juego['genres']])  # Concatenar géneros
                    })

            if not data.get('next'):  # Si no hay más páginas, terminar bucle
                break

            time.sleep(1)  # Esperar 1 segundo para no saturar la API (rate limit)

        # Si no se encontraron juegos válidos, añadir marcador 'Sin datos'
        if not juegos_este_año:
            juegos_por_año.append({
                'nombre': 'Sin datos',
                'fecha_lanzamiento': None,
                'año': año,
                'puntuacion_media': None,
                'generos': 'sin_datos'
            })
        else:
            juegos_por_año.extend(juegos_este_año)  # Añadir los juegos válidos al total

    return juegos_por_año  # Devolver la lista con todos los juegos

## 4️. Función para guardar los resultados en un archivo CSV

In [ ]:
def guardar_csv(datos, archivo='videojuegos_1995_2024_completo.csv'):
    '''Guarda los resultados en un archivo CSV con codificación UTF-8.'''
    with open(archivo, 'w', newline='', encoding='utf-8') as f:
        campos = ['nombre', 'fecha_lanzamiento', 'año', 'puntuacion_media', 'generos']
        writer = csv.DictWriter(f, fieldnames=campos)
        writer.writeheader()
        writer.writerows(datos)

## 5️. Ejecutar la descarga y guardar los datos

In [4]:
# guardar_csv(juegos)

## Aplicación de este notebook
- Recorre los años de 1995 a 2024.
- Recupera videojuegos con puntuación (> 0) y al menos un género.
- Guarda hasta 3 páginas por año (40 juegos por página).
- Inserta 'Sin datos' si un año no devuelve resultados válidos.
- Guarda todo en `videojuegos_1995_2024_completo.csv` para análisis posterior.